# Initial Exploration

## Imports

In [51]:
import pandas as pd
# from fuzzywuzzy import process
from collections import defaultdict
from rapidfuzz import process

## Read Data

In [52]:
match_info_df = pd.read_csv('//data/match_info.csv')
match_outcome_stats_df = pd.read_csv('//data/match_outcome_stats.csv')
player_info_df = pd.read_csv('//data/player_info.csv')
player_outcome_stats_df = pd.read_csv('//data/player_outcome_stats.csv')

## Match Info

In [53]:
match_info_df.head()

,match_id,tourney_id,tourney_name,tourney_date,tourney_level,surface,match_num,best_of,round
0,0,0,Orlando,2000-05-01,A,Clay,1,3.0,R32
1,1,0,Orlando,2000-05-01,A,Clay,2,3.0,R32
2,2,0,Orlando,2000-05-01,A,Clay,3,3.0,R32
3,3,0,Orlando,2000-05-01,A,Clay,4,3.0,R32
4,4,0,Orlando,2000-05-01,A,Clay,5,3.0,R32


Look at data types and unique values.

In [54]:
match_info_df.dtypes

match_id           int64
tourney_id         int64
tourney_name      object
tourney_date      object
tourney_level     object
surface           object
match_num          int64
best_of          float64
round             object
dtype: object

In [55]:
match_info_df.tourney_level.unique()

array(['A', 'G', 'M', 'F', 'C', 'D'], dtype=object)

In [56]:
match_info_df.surface.unique()

array(['Clay', 'Hard', 'Grass', 'Carpet'], dtype=object)

In [57]:
match_info_df['round'].unique()

array(['R32', 'R16', 'QF', 'SF', 'F', 'R128', 'R64', 'RR', 'BR'],
      dtype=object)

In [58]:
match_info_df.best_of.unique()

array([3., 5.])

Look at combination of (tourney_id, tourney_name). See if there is any tourney_id that has multiple tourney_name.

In [59]:
tourney_id_group = match_info_df.groupby('tourney_id')['tourney_name'].nunique()
print(f"Number of tourney_id with multiple tourney_name: {len(tourney_id_group[tourney_id_group > 1])}")

Number of tourney_id with multiple tourney_name: 0


Use string similarity packages to check for similar tourney_name. (Multiple choices here but using `fuzzywuzzy` for now)

In [60]:
sorted_lower_tourney_names = match_info_df['tourney_name'].str.lower().sort_values().unique()

In [61]:
sorted_lower_tourney_names

array(["'s-hertogenbosch", 'acapulco', 'adelaide', 'amersfoort',
       'amsterdam', 'antwerp', 'atlanta', 'auckland', 'australian open',
       'bangkok', 'barcelona', 'basel', 'bastad', 'beijing',
       'beijing olympics', 'belgrade', 'bogota', 'brighton', 'brisbane',
       'bucharest', 'buenos aires', 'canada masters', 'casablanca',
       'chengdu', 'chennai', 'cincinnati masters', 'copenhagen',
       'costa do sauipe', 'curitiba ch', 'davis cup g1 r1: bar vs ecu',
       'davis cup g1 r1: chi vs dom', 'davis cup g1 r1: dom vs chi',
       'davis cup g1 r1: hun vs isr', 'davis cup g1 r1: isr vs por',
       'davis cup g1 r1: nzl vs ind', 'davis cup g1 r1: nzl vs kor',
       'davis cup g1 r1: pak vs chn', 'davis cup g1 r1: per vs ecu',
       'davis cup g1 r1: pol vs bih', 'davis cup g1 r1: por vs aut',
       'davis cup g1 r1: rou vs blr', 'davis cup g1 r1: rou vs slo',
       'davis cup g1 r1: rus vs swe', 'davis cup g1 r1: uzb vs kor',
       'davis cup g1 r2: bra vs ecu', 'd

We see a lot of repeated "davis cup" tournaments where the tournament name contains round and competitor information.  We will exclude these from our analysis.

In [62]:
match_info_df = match_info_df[~match_info_df['tourney_name'].str.contains('davis cup', case=False)]
sorted_lower_tourney_names = match_info_df['tourney_name'].str.lower().sort_values().unique()

Set similarity threshold to 80

In [63]:
CHOSEN_THRESHOLD = 80

In [64]:
similarity_results = []
for name in sorted_lower_tourney_names:
    candidates = [n for n in sorted_lower_tourney_names if n != name]
    matches = process.extract(name, candidates, limit=len(candidates))
    
    for match in matches:
        if match[1] >= CHOSEN_THRESHOLD:
            similarity_results.append(
                {
                    'name': name,
                    'match': match[0],
                    'score': match[1]
                }
            )

In [65]:
similarity_results

[{'name': "'s-hertogenbosch",
  'match': 's-hertogenbosch',
  'score': 96.7741935483871},
 {'name': 'australian open', 'match': 'us open', 'score': 85.5},
 {'name': 'beijing', 'match': 'beijing olympics', 'score': 90.0},
 {'name': 'beijing olympics', 'match': 'beijing', 'score': 90.0},
 {'name': 'beijing olympics', 'match': 'olympics', 'score': 90.0},
 {'name': 'cincinnati masters', 'match': 'masters cup', 'score': 85.5},
 {'name': 'cincinnati masters', 'match': 'rome masters', 'score': 85.5},
 {'name': 'hamburg', 'match': 'hamburg masters', 'score': 90.0},
 {'name': 'hamburg masters', 'match': 'hamburg', 'score': 90.0},
 {'name': 'indian wells masters', 'match': 'masters cup', 'score': 85.5},
 {'name': 'indian wells masters', 'match': 'miami masters', 'score': 85.5},
 {'name': 'indian wells masters', 'match': 'paris masters', 'score': 85.5},
 {'name': 'indian wells masters', 'match': 'rome masters', 'score': 85.5},
 {'name': 'london', 'match': 'london olympics', 'score': 90.0},
 {'nam

### Action

- There are a lot of close "matches" that look genuine, i.e, containing extra spaces or foreign characters. These should be mapped to the correct name & id.
- There are some that require further manual inspection.
- The Davis cup entries require further processing and cleaning.

## Match Outcome Stats

In [66]:
match_outcome_stats_df.head()

,match_id,score,minutes
0,0,3-6 7-6(6) 7-6(4),162.0
1,1,6-2 7-5,86.0
2,2,6-1 6-3,64.0
3,3,4-6 6-2 7-5,150.0
4,4,6-1 6-4,60.0


In [99]:
def parse_scores(score):
    sets = score.split(' ')
    parsed_sets = {}
    match_ended = False

    # Filter out any empty strings resulting from splitting the score
    sets = [set_score for set_score in sets if set_score.strip()]

    try:
        for i, set_score in enumerate(sets, 1):
            set_score = set_score.strip()  # Strip any leading/trailing whitespace
            if set_score in ['RET', 'DEF','W/O']:
                parsed_sets[f'set_{i}_p1'] = set_score
                parsed_sets[f'set_{i}_p2'] = set_score
                parsed_sets[f'set_{i}_winner'] = set_score
                match_ended = True
                break
            else:
                tiebreaker = None
                if '(' in set_score:
                    set_score, tiebreaker = set_score.split('(')
                    set_score = set_score.strip()
                    tiebreaker = tiebreaker.replace(')', '').strip()
                p1, p2 = set_score.split('-')
                parsed_sets[f'set_{i}_p1'] = int(p1)
                parsed_sets[f'set_{i}_p2'] = int(p2)
                if tiebreaker:
                    parsed_sets[f'set_{i}_tiebreaker'] = int(tiebreaker)
                parsed_sets[f'set_{i}_winner'] = 'p1' if int(p1) > int(p2) else 'p2'
        
        # Ensure all five sets are represented and handle cases where match ends early
        for i in range(1, 5):
            if f'set_{i}_p1' not in parsed_sets:
                parsed_sets[f'set_{i}_p1'] = 'RET' if match_ended and i > len(sets) else None
            if f'set_{i}_p2' not in parsed_sets:
                parsed_sets[f'set_{i}_p2'] = 'RET' if match_ended and i > len(sets) else None
            if f'set_{i}_winner' not in parsed_sets:
                parsed_sets[f'set_{i}_winner'] = 'RET' if match_ended and i > len(sets) else None
            if f'set_{i}_tiebreaker' not in parsed_sets:
                parsed_sets[f'set_{i}_tiebreaker'] = None

        # Determine if the match was completed
        parsed_sets['match_completed'] = not match_ended

    except Exception as e:
        print(f"Error parsing score '{score}': {e}")
        return None
    
    return parsed_sets

def parse_scores(score):
    sets = score.split(' ')
    parsed_sets = {}
    match_ended = False
    completion_status = True

    # Filter out any empty strings resulting from splitting the score
    sets = [set_score.strip() for set_score in sets if set_score.strip()]

    try:
        for i, set_score in enumerate(sets, 1):
            if set_score in {'RET', 'DEF', 'W/O'}:
                parsed_sets[f'set_{i}_p1'] = set_score
                parsed_sets[f'set_{i}_p2'] = set_score
                parsed_sets[f'set_{i}_winner'] = set_score
                match_ended = True
                completion_status = False
                break
            else:
                tiebreaker = None
                if '(' in set_score:
                    set_score, tiebreaker = set_score.split('(')
                    set_score = set_score.strip()
                    tiebreaker = tiebreaker.replace(')', '').strip()
                p1, p2 = map(int, set_score.split('-'))
                parsed_sets[f'set_{i}_p1'] = p1
                parsed_sets[f'set_{i}_p2'] = p2
                if tiebreaker:
                    parsed_sets[f'set_{i}_tiebreaker'] = int(tiebreaker)
                parsed_sets[f'set_{i}_winner'] = 'p1' if p1 > p2 else 'p2'
        
        # Ensure all five sets are represented and handle cases where match ends early
        for i in range(1, 5):
            if f'set_{i}_p1' not in parsed_sets:
                parsed_sets[f'set_{i}_p1'] = 'RET' if match_ended else None
            if f'set_{i}_p2' not in parsed_sets:
                parsed_sets[f'set_{i}_p2'] = 'RET' if match_ended else None
            if f'set_{i}_winner' not in parsed_sets:
                parsed_sets[f'set_{i}_winner'] = 'RET' if match_ended else None
            if f'set_{i}_tiebreaker' not in parsed_sets:
                parsed_sets[f'set_{i}_tiebreaker'] = None

        # Determine if the match was completed
        parsed_sets['match_completed'] = completion_status

    except Exception as e:
        print(f"Error parsing score '{score}': {e}")
        return None
    
    return parsed_sets

# Define a function to parse the scores and determine the winner for each set
def parse_scores(score):
    sets = score.split(' ')
    parsed_sets = {}
    match_ended = False
    completion_status = True

    # Filter out any empty strings resulting from splitting the score
    sets = [set_score.strip() for set_score in sets if set_score.strip()]

    p1_set_wins = 0
    p2_set_wins = 0

    try:
        for i, set_score in enumerate(sets, 1):
            if set_score in {'RET', 'DEF', 'W/O'}:
                parsed_sets[f'set_{i}_p1'] = set_score
                parsed_sets[f'set_{i}_p2'] = set_score
                parsed_sets[f'set_{i}_winner'] = set_score
                match_ended = True
                completion_status = False
                break
            else:
                tiebreaker = None
                if '(' in set_score:
                    set_score, tiebreaker = set_score.split('(')
                    set_score = set_score.strip()
                    tiebreaker = tiebreaker.replace(')', '').strip()
                p1, p2 = map(int, set_score.split('-'))
                parsed_sets[f'set_{i}_p1'] = p1
                parsed_sets[f'set_{i}_p2'] = p2
                if tiebreaker:
                    parsed_sets[f'set_{i}_tiebreaker'] = int(tiebreaker)
                if p1 > p2:
                    parsed_sets[f'set_{i}_winner'] = 'p1'
                    p1_set_wins += 1
                else:
                    parsed_sets[f'set_{i}_winner'] = 'p2'
                    p2_set_wins += 1
        
        # Ensure all five sets are represented and handle cases where match ends early
        for i in range(1, 5):
            if f'set_{i}_p1' not in parsed_sets:
                parsed_sets[f'set_{i}_p1'] = 'RET' if match_ended else None
            if f'set_{i}_p2' not in parsed_sets:
                parsed_sets[f'set_{i}_p2'] = 'RET' if match_ended else None
            if f'set_{i}_winner' not in parsed_sets:
                parsed_sets[f'set_{i}_winner'] = 'RET' if match_ended else None
            if f'set_{i}_tiebreaker' not in parsed_sets:
                parsed_sets[f'set_{i}_tiebreaker'] = None

        # Determine if the match was completed
        parsed_sets['match_completed'] = completion_status

        # Determine the overall winner
        if completion_status:
            if p1_set_wins > p2_set_wins:
                parsed_sets['match_winner'] = 'p1'
            else:
                parsed_sets['match_winner'] = 'p2'

    except Exception as e:
        print(f"Error parsing score '{score}': {e}")
        return None
    
    return parsed_sets
# Apply the parsing function to the score column
parsed_scores = match_outcome_stats_df['score'].apply(parse_scores)

# Remove rows with parsing errors
df = match_outcome_stats_df[parsed_scores.notnull()]

# Convert the parsed scores to a DataFrame and merge with the original DataFrame
parsed_scores_df = pd.DataFrame(parsed_scores.dropna().tolist())
df = df.join(parsed_scores_df)

# Drop the original score column
df.drop(columns=['score'], inplace=True)

Error parsing score '3-Jun': invalid literal for int() with base 10: 'Jun'


In [111]:

import pandas as pd
from typing import Any, Dict, Optional, Union

def parse_set_score(set_score: str) -> Dict[str, Union[int, str, None]]:
    """
    Parse a single set score and extract points for player 1 and player 2,
    as well as tiebreaker points if present.

    Parameters:
    - set_score (str): The set score to parse.

    Returns:
    - Dict[str, Union[int, str, None]]: A dictionary containing the parsed set score details.
    """
    set_score = set_score.strip()
    if set_score in {'RET', 'DEF', 'W/O'}:
        return {
            'p1': set_score,
            'p2': set_score,
            'winner': set_score,
            'tiebreaker': None
        }
    tiebreaker = None
    if '(' in set_score:
        set_score, tiebreaker = set_score.split('(')
        set_score = set_score.strip()
        tiebreaker = int(tiebreaker.replace(')', '').strip())
    p1, p2 = map(int, set_score.split('-'))
    winner = 'p1' if p1 > p2 else 'p2'
    return {
        'p1': p1,
        'p2': p2,
        'winner': winner,
        'tiebreaker': tiebreaker
    }

def parse_scores(score: str) -> Optional[Dict[str, Any]]:
    """
    Parse the match score string into a structured format.

    Parameters:
    - score (str): The score string to parse.

    Returns:
    - Optional[Dict[str, Any]]: A dictionary containing the parsed match details,
      or None if the parsing failed.
    """
    sets = [set_score.strip() for set_score in score.split(' ') if set_score.strip()]
    parsed_sets: Dict[str, Any] = {}
    match_ended = False
    p1_set_wins = 0
    p2_set_wins = 0

    try:
        for i, set_score in enumerate(sets, 1):
            if i>5:
                print(f"Error parsing score '{score}': More than 5 sets")
            set_result = parse_set_score(set_score)
            parsed_sets[f'set_{i}_p1'] = set_result['p1']
            parsed_sets[f'set_{i}_p2'] = set_result['p2']
            parsed_sets[f'set_{i}_tiebreaker'] = set_result['tiebreaker']
            parsed_sets[f'set_{i}_winner'] = set_result['winner']
            if set_result['winner'] in {'RET', 'DEF', 'W/O'}:
                match_ended = True
                break
            elif set_result['winner'] == 'p1':
                p1_set_wins += 1
            elif set_result['winner'] == 'p2':
                p2_set_wins += 1

        # Ensure all five sets are represented and handle cases where match ends early
        for i in range(1, 5):
            parsed_sets.setdefault(f'set_{i}_p1', 'RET' if match_ended else None)
            parsed_sets.setdefault(f'set_{i}_p2', 'RET' if match_ended else None)
            parsed_sets.setdefault(f'set_{i}_winner', 'RET' if match_ended else None)
            parsed_sets.setdefault(f'set_{i}_tiebreaker', None)

        # Determine if the match was completed
        parsed_sets['match_completed'] = not match_ended

        # Determine the overall winner if the match was completed
        if not match_ended:
            if p1_set_wins > p2_set_wins:
                parsed_sets['match_winner'] = 'p1'
            elif p2_set_wins > p1_set_wins:
                parsed_sets['match_winner'] = 'p2'
            else:
                parsed_sets['match_winner'] = None
        else:
            parsed_sets['match_winner'] = None

    except Exception as e:
        print(f"Error parsing score '{score}': {e}")
        return None

    return parsed_sets

def process_match_scores(df: pd.DataFrame) -> pd.DataFrame:
    """
    Process the match scores in the DataFrame and add structured match details.

    Parameters:
    - df (pd.DataFrame): The DataFrame containing match scores.

    Returns:
    - pd.DataFrame: The DataFrame with structured match details added.
    """
    parsed_scores = df['score'].apply(parse_scores)
    
    # Remove rows with parsing errors
    parsed_scores = parsed_scores.dropna()
    
    # Convert the parsed scores to a DataFrame and merge with the original DataFrame
    parsed_scores_df = pd.DataFrame(parsed_scores.tolist())
    df = df.loc[parsed_scores.index].join(parsed_scores_df)
    df.drop(columns=['score'], inplace=True)
    return df


# Process the match scores
df = process_match_scores(match_outcome_stats_df)

# Display the resulting DataFrame
print(df)



Error parsing score '4-6 4-6 6-3 7-5 3-4 RET': More than 5 sets
Error parsing score '4-6 6-2 6-2 4-6 0-1 RET': More than 5 sets
Error parsing score '7-5 4-6 5-7 6-3 5-0 RET': More than 5 sets
Error parsing score '6-4 6-2 6-7(5) 3-6 2-1 RET': More than 5 sets
Error parsing score '2-6 6-7(3) 6-4 7-6(4) 0-1 RET': More than 5 sets
Error parsing score '4-6 6-3 7-6(3) 5-7 3-2 RET': More than 5 sets
Error parsing score '7-6(4) 4-6 3-6 7-6(9) 1-0 RET': More than 5 sets
Error parsing score '1-6 6-3 3-6 7-6(5) 4-1 RET': More than 5 sets
Error parsing score '6-7(5) 6-3 6-1 6-7(4) 3-0 RET': More than 5 sets
Error parsing score '4-6 6-3 3-6 6-0 3-0 RET': More than 5 sets
Error parsing score '6-7(5) 4-6 7-6(2) 6-0 2-1 RET': More than 5 sets
Error parsing score '7-6(7) 4-6 6-7(3) 6-4 4-0 RET': More than 5 sets
Error parsing score '3-Jun': invalid literal for int() with base 10: 'Jun'
Error parsing score '6-7(7) 6-7(3) 7-6(7) 6-3 3-0 RET': More than 5 sets
Error parsing score '6-2 3-6 4-6 7-6(2) 5-4 R

In [104]:
df = match_outcome_stats_df.copy()
# Apply the parsing function to the score column
parsed_scores = df['score'].apply(parse_scores)
parsed_scores_df = pd.DataFrame(parsed_scores.dropna().tolist())

Error parsing score '3-Jun': invalid literal for int() with base 10: 'Jun'


In [91]:
df['match_completed'].value_counts()

match_completed
True     46199
False     1539
Name: count, dtype: int64

### Issues

- We don't know the rules so for tiebreaks, we don't know how many points the winning player obtained, i.e. 7, 10 etc.
- We don't know which player Retires or Defaults.
- We don't know rules on final set so cannot do any validation, i.e. is final set a tiebreak or continue until 2 game lead.